# What Is Hyperparameter Tuning?
- Hyperparameters are settings you define before training a model (e.g., learning_rate, max_depth, n_estimators in tree-based models).
- Hyperparameter Tuning is the process of finding the best combination of hyperparameters to improve model performance.

### When to Tune?
- When you want to improve performance beyond default parameters.
- After you've done feature engineering and have a stable baseline model.
- When the model is underfitting or overfitting.

### Popular Tuning Techniques

**1. Grid Search**

**Explanation:**
- Grid search tries every possible combination of hyperparameters from a given list. It's exhaustive.

**Real-World Use Case:**
- You’re tuning a LightGBM model and want to test all combinations of max_depth and learning_rate.

In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report

# Load classification dataset
data = load_breast_cancer()
X = data.data
y = data.target

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define model
rf = RandomForestClassifier(random_state=42)

# Define hyperparameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid,
                           scoring='accuracy', cv=5, verbose=1, n_jobs=-1)

# Fit the model
grid_search.fit(X_train, y_train)

# Print best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best CV Score:", grid_search.best_score_)

# Evaluate on test data
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

print("\nClassification Report on Test Data:\n")
print(classification_report(y_test, y_pred))

Fitting 5 folds for each of 36 candidates, totalling 180 fits
Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Best CV Score: 0.9626373626373625

Classification Report on Test Data:

              precision    recall  f1-score   support

           0       0.98      0.93      0.95        43
           1       0.96      0.99      0.97        71

    accuracy                           0.96       114
   macro avg       0.97      0.96      0.96       114
weighted avg       0.97      0.96      0.96       114



**When to Use:**
- Small dataset
- Few hyperparameters
- You want to be thorough

#### Pros & Cons:

| Pros                     | Cons                                  |
|--------------------------|----------------------------------------|
| Simple and Systematic  | Computationally expensive                |
| Finds best combo | Time-consuming for large grids |

**2. RANDOM SEARCH**
**Explanation:**
- Instead of testing all combinations, it picks random combinations. Much faster.

**Real-World Use Case:**
- You're testing a deep learning model with many layers, and it's too expensive to try every combo.

In [3]:
from sklearn.datasets import load_wine
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
import numpy as np

# Load the Wine dataset
data = load_wine()
X = data.data
y = data.target

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model
rf = RandomForestClassifier(random_state=42)

# Define hyperparameter distributions
param_dist = {
    'n_estimators': randint(50, 200),
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': randint(2, 11),
    'min_samples_leaf': randint(1, 5)
}

# Setup RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=rf,
                                   param_distributions=param_dist,
                                   n_iter=20,
                                   scoring='accuracy',
                                   cv=5,
                                   verbose=1,
                                   random_state=42,
                                   n_jobs=-1)

# Fit the model
random_search.fit(X_train, y_train)

# Show best results
print("Best Parameters:", random_search.best_params_)
print("Best CV Score:", random_search.best_score_)

# Evaluate on test set
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)

print("\nClassification Report on Test Data:\n")
print(classification_report(y_test, y_pred))

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best Parameters: {'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 98}
Best CV Score: 0.9785714285714286

Classification Report on Test Data:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        14
           1       1.00      1.00      1.00        14
           2       1.00      1.00      1.00         8

    accuracy                           1.00        36
   macro avg       1.00      1.00      1.00        36
weighted avg       1.00      1.00      1.00        36



**When to Use:**
- Large hyperparameter space
- Limited time
- Rough estimate is okay

#### Pros & Cons:

| Pros                     | Cons                                  |
|--------------------------|---------------------------------------|
| Faster than grid search  | Might miss best combo                |
| Better for large hyperparameter space  | Still requires manual ranges |

**3. BAYESIAN OPTIMIZATION (using Optuna)**

**Explanation:**
- It learns from previous results and selects the next best hyperparameter set to try — smart searching.

**Real-World Use Case:**
- You're tuning a neural network or a complex ML model, and want a smart, automated way to improve over time.

In [4]:
!pip install optuna


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: C:\Users\hp\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.1 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.1 MB ? eta -:--:--
   --------- ------------------------------ 0.5/2.1 MB 670.4 kB/s eta 0:00:03
   -------------- ------------------------- 0.8/2.1 MB 744.7 kB/s eta 0:00:02
   -------------- ------------------------- 0.8/2.1 MB 744.7 kB/s eta 0:00:02
   ------------------- -------------------- 1.0/2.1 MB 739.8 kB/s eta 0:00:02
   ------------------- -------------------- 1.0/2.1 MB 739.8 kB/s eta 0:00:02
   ------------------------ --------------- 1.3/2.1 MB 684.4 kB/s eta 0:00:02
   ------------------------ --------------- 1.3/2.1 MB 684.4 kB/s eta 0:00:02
   ----------------------------- ---------- 1.6/2.1 MB 693.1 kB/s eta 0:00:01
   ---------------------------

In [5]:
import optuna
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import cross_val_score, StratifiedKFold

# Load the Breast Cancer dataset
data = load_breast_cancer()
X = data.data
y = data.target

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function for optimization
def objective(trial):
    # Hyperparameter search space
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 4)
    
    clf = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42
    )
    
    # Use cross-validation
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    score = cross_val_score(clf, X_train, y_train, cv=cv, scoring='accuracy').mean()
    return score

# Run Optuna study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)

# Show best results
print("Best Parameters:", study.best_params)
print("Best CV Score:", study.best_value)

# Train best model and evaluate on test set
best_model = RandomForestClassifier(**study.best_params, random_state=42)
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)

# Classification results
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))

[I 2025-06-03 09:33:17,359] A new study created in memory with name: no-name-b7bda316-d67e-47b8-af69-0ad24162c1de
[I 2025-06-03 09:33:19,811] Trial 0 finished with value: 0.9472527472527472 and parameters: {'n_estimators': 165, 'max_depth': 16, 'min_samples_split': 3, 'min_samples_leaf': 4}. Best is trial 0 with value: 0.9472527472527472.
[I 2025-06-03 09:33:20,792] Trial 1 finished with value: 0.9494505494505494 and parameters: {'n_estimators': 58, 'max_depth': 13, 'min_samples_split': 8, 'min_samples_leaf': 1}. Best is trial 1 with value: 0.9494505494505494.
[I 2025-06-03 09:33:22,132] Trial 2 finished with value: 0.9472527472527472 and parameters: {'n_estimators': 83, 'max_depth': 18, 'min_samples_split': 3, 'min_samples_leaf': 3}. Best is trial 1 with value: 0.9494505494505494.
[I 2025-06-03 09:33:23,392] Trial 3 finished with value: 0.9472527472527472 and parameters: {'n_estimators': 82, 'max_depth': 8, 'min_samples_split': 6, 'min_samples_leaf': 3}. Best is trial 1 with value: 0.

Best Parameters: {'n_estimators': 189, 'max_depth': 10, 'min_samples_split': 5, 'min_samples_leaf': 2}
Best CV Score: 0.956043956043956

Classification Report:

              precision    recall  f1-score   support

           0       0.98      0.93      0.95        43
           1       0.96      0.99      0.97        71

    accuracy                           0.96       114
   macro avg       0.97      0.96      0.96       114
weighted avg       0.97      0.96      0.96       114



#### Pros & Cons of Bayesian Optimization (Optuna)

| Pros                          | Cons                                |
|-------------------------------|--------------------------------------|
| Efficient, smart tuning       | Slightly complex setup               |
| Faster convergence than grid  | Needs external library (Optuna)     |
| Supports pruning and parallel | Harder to explain than GridSearchCV |

**When to Use:**
- Many hyperparameters
- You want efficient search
- Medium/large datasets

**4. HYPEROPT (TPE - Tree-structured Parzen Estimator)**

**Explanation:**
- Hyperopt uses TPE (a smarter version of Bayesian search) for hyperparameter tuning.

**Real-World Use Case:**
- You want to tune a model using a smarter-than-random method, but you want more control than Optuna.

In [6]:
!pip install hyperopt xgboost

   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.6 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.6 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.6 MB ? eta -:--:--
   ------------- -------------------------- 0.5/1.6 MB 372.9 kB/s eta 0:00:03
   ------------- -------------------------- 0.5/1.6 MB 372.9 kB/s eta 0:00:03
   ------------- -------------------------- 0.5/1.6 MB 372.9 kB/s eta 0:00:03
   ------------------- -------------------- 0.8/1.6 MB 419.4 kB/s eta 0:00:0


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: C:\Users\hp\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [7]:
from sklearn.datasets import load_digits
from xgboost import XGBClassifier
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK

# Load the Digits dataset
digits = load_digits()
X = digits.data
y = digits.target

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the search space
space = {
    'n_estimators': hp.choice('n_estimators', range(50, 300)),
    'max_depth': hp.choice('max_depth', range(3, 15)),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),
    'gamma': hp.uniform('gamma', 0, 5),
    'subsample': hp.uniform('subsample', 0.5, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0)
}

# Define objective function
def objective(params):
    model = XGBClassifier(**params, use_label_encoder=False, eval_metric='mlogloss', random_state=42)
    acc = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=5).mean()
    return {'loss': -acc, 'status': STATUS_OK}

# Run optimization
trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=30,
            trials=trials,
            rstate=np.random.default_rng(42))

print("\nBest Hyperparameters:\n", best)

# Rebuild the best model (convert integer indices back to actual values where needed)
best_model = XGBClassifier(
    n_estimators=best['n_estimators'],
    max_depth=best['max_depth'],
    learning_rate=best['learning_rate'],
    gamma=best['gamma'],
    subsample=best['subsample'],
    colsample_bytree=best['colsample_bytree'],
    use_label_encoder=False,
    eval_metric='mlogloss',
    random_state=42
)

# Fit and evaluate
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
print("\nClassification Report on Test Data:\n")
print(classification_report(y_test, y_pred))

  0%|                                                                           | 0/30 [00:00<?, ?trial/s, best loss=?]

C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\xgboost\training.py:183: UserWarning: [09:38:24] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)

C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\xgboost\training.py:183: UserWarning: [09:38:26] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)

C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\xgboost\training.py:183: UserWarning: [09:38:27] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not

  3%|█▌                                              | 1/30 [00:06<02:56,  6.10s/trial, best loss: -0.9366555361982192]

C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\xgboost\training.py:183: UserWarning: [09:38:30] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)

C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\xgboost\training.py:183: UserWarning: [09:38:31] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)

C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\xgboost\training.py:183: UserWarning: [09:38:32] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not

  7%|███▏                                            | 2/30 [00:10<02:22,  5.10s/trial, best loss: -0.9638066202090592]

...

 97%|█████████████████████████████████████████████▍ | 29/30 [02:25<00:05,  5.35s/trial, best loss: -0.9645034843205575]

C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\xgboost\training.py:183: UserWarning: [09:40:49] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)

C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\xgboost\training.py:183: UserWarning: [09:40:50] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)

C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\xgboost\training.py:183: UserWarning: [09:40:51] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not

100%|███████████████████████████████████████████████| 30/30 [02:29<00:00,  4.98s/trial, best loss: -0.9645034843205575]

Best Hyperparameters:
 {'colsample_bytree': 0.8240840478365372, 'gamma': 0.024295119711918223, 'learning_rate': 0.2165665569107049, 'max_depth': 6, 'n_estimators': 227, 'subsample': 0.5857046698603472}


C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\xgboost\training.py:183: UserWarning: [09:40:54] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



Classification Report on Test Data:

              precision    recall  f1-score   support

           0       1.00      0.97      0.98        33
           1       0.97      1.00      0.98        28
           2       1.00      1.00      1.00        33
           3       1.00      0.97      0.99        34
           4       0.98      1.00      0.99        46
           5       0.94      0.96      0.95        47
           6       0.97      0.97      0.97        35
           7       0.94      0.97      0.96        34
           8       1.00      0.93      0.97        30
           9       0.95      0.95      0.95        40

    accuracy                           0.97       360
   macro avg       0.97      0.97      0.97       360
weighted avg       0.97      0.97      0.97       360



#### Pros & Cons of Hyperopt (TPE)

| Pros                                | Cons                                |
|-------------------------------------|-------------------------------------|
| Smarter search than Grid/Random     | Not as fast as Optuna for some tasks |
| Supports conditional spaces         | Slightly more complex syntax         |
| Good with large search spaces       | Visualization tools not built-in     |

**When to Use:**
- You like low-level control
- Need a smart alternativeto grid/random
- You’re experimenting with research-style projects